# Economy in East Africa
***
<img src="https://images.unsplash.com/photo-1633158829875-e5316a358c6f?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=3570&q=80" alt="Money Jar" style="width:500px;">

<a id="WebScraping"><h2>Getting Data</h2></a>
***
<a id="ScraperImport"><h3>Importing Libraries</h3></a>

In [1]:

import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.precision", 2)

# Data Wrapper
import wbdata

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# styling
%matplotlib inline
sns.set_theme(style="dark")
mpl.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns',None)
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

<a id="indicatorspecific"><h3>Import Data from WBDATA API</h3></a>

In [2]:
df=wbdata.get_indicator()
print(df)

ChunkedEncodingError: ('Connection broken: IncompleteRead(293449 bytes read, 322217 more expected)', IncompleteRead(293449 bytes read, 322217 more expected))

In [ ]:
# Define the list of indicators
indicators = {
    "NE.EXP.GNFS.CD":"Exports of goods and services (current US$)",
    "BN.GSR.GNFS.CD":"Net trade in goods and services (BoP, current US$)",
    "NE.CON.TOTL.ZS":"Final consumption expenditure (% of GDP)",
    "BX.TRF.MGR.DT.GD.ZS":"Migrant remittance inflows (% of GDP)",
    "NY.GDP.MKTP.CD":"GDP (current US$)",
    "NY.GDP.MKTP.KD.ZG":"GDP growth (annual %)",
    "NY.GDP.PCAP.PP.CD":"GDP per capita, PPP (current international $)",
    "NY.GNP.MKTP.CD":"GNI (current US$)",
    "NE.IMP.GNFS.CD":"Imports of goods and services (current US$)"
   

}

# Define the list of countries
country_codes = ["SOM", "KEN", "ETH", "SDN", "SSD", "ERI", "DJI", "RWA", "UGA", "BDI", "TZA",'WLD']

# Get the data as a pandas DataFrame
df=wbdata.get_dataframe(indicators,country=country_codes,convert_date=True, keep_levels=True)

# Reset the index and rename the columns
df = df.reset_index().rename(columns={"country": "Country", "date": "Year"})

df.to_csv('east_africa_economy_data.csv',index=False, header=True)
# Display the DataFrame
print(df)

     Country       Year  Exports of goods and services (current US$)  \
0    Burundi 1960-01-01                                     2.45e+07   
1    Burundi 1961-01-01                                     2.62e+07   
2    Burundi 1962-01-01                                     2.80e+07   
3    Burundi 1963-01-01                                     1.75e+07   
4    Burundi 1964-01-01                                     3.67e+07   
..       ...        ...                                          ...   
751    World 2018-01-01                                     2.52e+13   
752    World 2019-01-01                                     2.48e+13   
753    World 2020-01-01                                     2.25e+13   
754    World 2021-01-01                                     2.80e+13   
755    World 2022-01-01                                     3.08e+13   

     Net trade in goods and services (BoP, current US$)  \
0                                                  NaN    
1                

<a id="EDA"><h2>Exploratory Data Analysis</h2></a>
***
<a id="structure"><h3>Checking Data Structure</h3></a>

In [ ]:
df=pd.read_csv('east_africa_economy_data.csv')
df.rename(columns={'Country Name':'Country'},inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 11 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Country                                             756 non-null    object 
 1   Year                                                756 non-null    object 
 2   Exports of goods and services (current US$)         490 non-null    float64
 3   Net trade in goods and services (BoP, current US$)  321 non-null    float64
 4   Final consumption expenditure (% of GDP)            425 non-null    float64
 5   Migrant remittance inflows (% of GDP)               199 non-null    float64
 6   GDP (current US$)                                   561 non-null    float64
 7   GDP growth (annual %)                               499 non-null    float64
 8   GDP per capita, PPP (current international $)       292 non-null    float64
 9  

In [ ]:
print(df.shape)
df.tail(3)

(756, 11)


,Country,Year,Exports of goods and services (current US$),"Net trade in goods and services (BoP, current US$)",Final consumption expenditure (% of GDP),Migrant remittance inflows (% of GDP),GDP (current US$),GDP growth (annual %),"GDP per capita, PPP (current international $)",GNI (current US$),Imports of goods and services (current US$)
753,World,2020-01-01,2.25e+13,NaN,NaN,NaN,8.52e+13,-3.07,17219.30,8.53e+13,2.18e+13
754,World,2021-01-01,2.80e+13,NaN,NaN,NaN,9.69e+13,6.02,18780.78,9.68e+13,2.71e+13
755,World,2022-01-01,3.08e+13,NaN,NaN,NaN,1.01e+14,3.08,20645.48,1.00e+14,3.03e+13


In [ ]:
# Separate 'World' data and calculate mean for East African Countries
world_data = df[df['Country'] == 'World'].round(2)
east_africa_countries_data = df[df['Country'] != 'World']
world_data.columns = [f"{col} (World)" if col != 'Year' else col for col in world_data.columns]
world_data

,Country (World),Year,Exports of goods and services (current US$) (World),"Net trade in goods and services (BoP, current US$) (World)",Final consumption expenditure (% of GDP) (World),Migrant remittance inflows (% of GDP) (World),GDP (current US$) (World),GDP growth (annual %) (World),"GDP per capita, PPP (current international $) (World)",GNI (current US$) (World),Imports of goods and services (current US$) (World)
693,World,1960-01-01,NaN,NaN,NaN,NaN,1.38e+12,NaN,NaN,1.37e+12,NaN
694,World,1961-01-01,NaN,NaN,NaN,NaN,1.45e+12,3.79,NaN,1.44e+12,NaN
695,World,1962-01-01,NaN,NaN,NaN,NaN,1.55e+12,5.31,NaN,1.54e+12,NaN
696,World,1963-01-01,NaN,NaN,NaN,NaN,1.67e+12,5.18,NaN,1.65e+12,NaN
697,World,1964-01-01,NaN,NaN,NaN,NaN,1.83e+12,6.56,NaN,1.82e+12,NaN
...,...,...,...,...,...,...,...,...,...,...,...
751,World,2018-01-01,2.52e+13,NaN,NaN,NaN,8.65e+13,3.29,16974.70,8.66e+13,2.47e+13
752,World,2019-01-01,2.48e+13,NaN,NaN,NaN,8.77e+13,2.59,17628.44,8.78e+13,2.44e+13
753,World,2020-01-01,2.25e+13,NaN,NaN,NaN,8.52e+13,-3.07,17219.30,8.53e+13,2.18e+13
754,World,2021-01-01,2.80e+13,NaN,NaN,NaN,9.69e+13,6.02,18780.78,9.68e+13,2.71e+13


<a id="structure"><h3> Data Summary</h3></a>

In [ ]:
# Drop the 'Country' column and calculate summary statistics
summary_df = df.drop(columns=['Country']).describe().transpose()

# Rename the columns for better clarity
summary_df.rename(columns={'mean': 'Average', 'min': 'Minimum', 'max': 'Maximum'}, inplace=False)

# Transpose the summary DataFrame to have statistics as rows
summary_df = summary_df.transpose()

summary_df.style.set_caption("Summary Statistics").\
    set_table_styles([{'selector': 'caption',
                       'props': [('color', 'blue'),
                                 ('font-size', '18px')]}]).\
    background_gradient(cmap='Blues').\
    highlight_max(axis=0).\
    highlight_min(axis=0).\
    highlight_max(color='red').\
    highlight_min(color='green')

# Display the stylized summary dataframe
summary_df

,Exports of goods and services (current US$),"Net trade in goods and services (BoP, current US$)",Final consumption expenditure (% of GDP),Migrant remittance inflows (% of GDP),GDP (current US$),GDP growth (annual %),"GDP per capita, PPP (current international $)",GNI (current US$),Imports of goods and services (current US$)
count,4.90e+02,3.21e+02,425.00,1.99e+02,5.61e+02,499.00,292.00,5.53e+02,4.90e+02
mean,1.08e+12,-1.69e+09,93.76,1.64e+00,3.65e+12,3.98,2931.34,3.69e+12,1.06e+12
std,4.30e+12,2.64e+09,16.15,1.86e+00,1.43e+13,6.39,3549.45,1.44e+13,4.20e+12
min,7.60e+06,-1.41e+10,51.31,4.81e-04,1.19e+08,-50.25,313.60,1.19e+08,1.20e+07
25%,1.39e+08,-1.75e+09,84.74,1.77e-01,1.09e+09,1.71,886.12,1.10e+09,3.15e+08
50%,6.74e+08,-6.49e+08,92.31,7.27e-01,5.61e+09,4.40,1852.68,5.58e+09,1.32e+09
75%,4.94e+09,-2.52e+08,98.95,2.55e+00,2.70e+10,6.77,3109.65,2.71e+10,6.84e+09
max,3.08e+13,1.33e+09,157.54,9.49e+00,1.01e+14,35.22,20645.48,1.00e+14,3.03e+13


<a id="structure"><h3>Functions for Plots</h3></a>

In [ ]:
import numpy as np
import plotly.graph_objs as go

def create_line_chart_static(df_grouped, title, columns,colors,column="People"):
    # Create a trace for each variable
    traces = []
    variable = columns[0]
    category = columns[1]
  
    unique_countries = df_grouped["Country"].unique()
    frames = []

    for i, country in enumerate(unique_countries):
        color = colors[i % len(colors)]  # Cycle through colors for each country
        trace = go.Scatter(
            x=df_grouped[df_grouped["Country"] == country]['Year'],
            y=df_grouped[df_grouped["Country"] == country][variable],
            mode='lines',
            name=country,
            line=dict(color=color)
        )
        traces.append(trace)


   # Create the layout
    layout = go.Layout(
        autosize=True,
        width=900,
        height=700,
        title=title,
        xaxis=dict(title='Year', showgrid=False),
        yaxis=dict(title=column, showgrid=True),
        plot_bgcolor='#f8f8f8',
        paper_bgcolor='#f8f8f8',
        font=dict(family='Poppins', size=12, color='black'),
        legend=dict(x=0.10, y=0.8, bgcolor='rgba(255, 255, 255, 0.8)', bordercolor='rgba(0, 0, 0, 0.2)', borderwidth=1),
        margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad = 4
    )
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)

    # Add animation frames
    fig.update(frames=frames)



    
    # Apply Style
    fig.update_layout(
         annotations=[
            {
                'x': 0,
                'y': -0.06,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'left',
                'yanchor': 'top',
                'text': "Copyright © 2023 Najib Abdullahi",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "left",
            },
            {
                'x': 1.06,
                'y': -0.06,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'right',
                'yanchor': 'top',
                'text': "Source: World Bank",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "right",
            }
        ],
        font=dict(family='Poppins'),
        title_font=dict(size=20, color='#000000'),
        legend_font=dict(size=12),
        legend_itemclick='toggle',
        legend_itemdoubleclick='toggleothers',
        legend_traceorder='normal',
        legend_tracegroupgap=10,
        legend_bordercolor='#000000',
        legend_borderwidth=1,
        legend_bgcolor='#ffffff',
        legend_xanchor='left',
        legend_yanchor='top',
        legend_x=1.05,
        legend_y=1.05,
        margin=dict(l=50, r=50, t=80, b=50),
    )
    fig.update_yaxes(gridcolor='#c9c1b8')
    # fig.update_layout(yaxis=dict(range=[0,60]))
   

    # Display the plot
    fig.show()

In [ ]:
def create_line_chart_grouped(df_grouped,title,columns,colors,value='Percentage'):
    # create a trace for each variable
    traces=[]
    variables=columns
    

    for variable, color in zip(variables, colors):
        trace = go.Scatter(
            x=df_grouped['Year'],
            y=df_grouped[variable],
            mode='lines',
            name=variable,
            line=dict(color=color, width=2),
        )
        traces.append(trace)
    

    # Create the layout
    layout = go.Layout(
        autosize=True,
        width=700,
        height=700,
        title=title,
        xaxis=dict(title='Year', showgrid=False),
        yaxis=dict(title=value, showgrid=True),
        plot_bgcolor='#f8f8f8',
        paper_bgcolor='#f8f8f8',
        font=dict(family='Poppins', size=12, color='black'),
        legend=dict(x=0.10, y=0.8, bgcolor='rgba(255, 255, 255, 0.8)', bordercolor='rgba(0, 0, 0, 0.2)', borderwidth=1),
        margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad = 4
    )
    )

    # Create the figure and add the traces
    fig = go.Figure(data=traces, layout=layout)

    # Apply Style
    fig.update_layout(
        font=dict(family='Poppins'),
        title_font=dict(size=14, color='#000000'),
        legend_font=dict(size=12),
        legend_itemclick='toggle',
        legend_itemdoubleclick='toggleothers',
        legend_traceorder='normal',
        legend_tracegroupgap=10,
        legend_bordercolor='#000000',
        legend_borderwidth=1,
        legend_bgcolor='#ffffff',
        legend_xanchor='left',
        legend_yanchor='top',
        legend_x=0.9,
        legend_y=1.12,
        margin=dict(l=50, r=50, t=80, b=50),
    )
    fig.update_yaxes(gridcolor='#c9c1b8') 
  
   # Add annotation for copyright and source data
    fig.add_annotation(
        text="Copyright © 2023 Najib Abdullahi",
        showarrow=False,
        xref="paper",
        yref="paper",
        x=0.03,
        y=-0.07,
        xanchor="left",
        yanchor="top",
        font=dict(size=10, color="#1100FF"),
        align="left",
    )

    fig.add_annotation(
        text="Source: World Bank",
        showarrow=False,
        xref="paper",
        yref="paper",
        x=1,
        y=-0.07,
        xanchor="right",
        yanchor="top",
        font=dict(size=10, color="#1100FF"),
        align="right",
    )
    
    # Display the plot
    fig.show()

In [ ]:
def plot_horizontal_bar(dataframe, colors,title,countries,value,orientation="h"):

    
    
    # Sort countriesin descending order
    sorted_countries = sorted(countries, key=lambda country: dataframe[country], reverse=False)
    
    traces = []
    for country, color in zip(sorted_countries, colors):
        trace = {
            "type": "bar",
            "x": [dataframe[country]],
            "y": [country],
            "orientation": orientation,
            "marker": {"color": color},
            "name": country  # Adding the country name as a label for the marker
        }
        traces.append(trace)
    
    layout = {
        "title": title,
        "margin": {"l": 100, "r": 20, "t": 50, "b": 50},
        "barmode": "group",  
        'autosize': True,
        'width': 900,
        'height': 700,
        'xaxis': {'title': value,'showgrid': True, 'gridcolor': '#75A3BF'},
        'yaxis': {'title': 'Country', 'showgrid': False},
        'plot_bgcolor': '#f8f8f8',
        'paper_bgcolor': '#f8f8f8',
        'font': {'family': 'Poppins', 'size': 12, 'color': 'black'},
        'legend': {'x': 0.05, 'y': 0.5, 'bgcolor': 'rgba(255, 255, 255, 0.8)', 'bordercolor': 'rgba(0, 0, 0, 0.2)', 'borderwidth': 1},
        'margin': {'l': 50, 'r': 50, 'b': 100, 't': 100, 'pad': 4},

    }
    
    fig = go.Figure(data=traces, layout=layout)
    fig.update_layout(
        annotations=[
            {
                'x': 0,
                'y': -0.06,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'left',
                'yanchor': 'top',
                'text': "Copyright © 2023 Najib Abdullahi",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "left",
            },
            {
                'x': 1.06,
                'y': -0.06,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'right',
                'yanchor': 'top',
                'text': "Source: World Bank",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "right",
            }
        ],
        font=dict(family='Poppins'),
        title_font=dict(size=15, color='#000000'),
        legend_font=dict(size=12),
        legend_itemclick='toggle',
        legend_itemdoubleclick='toggleothers',
        legend_traceorder='normal',
        legend_tracegroupgap=10,
        legend_bordercolor='#000000',
        legend_borderwidth=1,
        legend_bgcolor='#ffffff',
        legend_xanchor='left',
        legend_yanchor='top',
        legend_x=1,
        legend_y=1.05,
        margin=dict(l=50, r=50, t=80, b=50),
    )
    fig.update_yaxes(gridcolor='#c9c1b8')

    fig.show()


In [ ]:
def create_stacked_bar_chart(v1,v2,years,title,value,v1_name,v2_name):
 
    
    fig = go.Figure()

    # Adding traces for Land area and Arable land
    fig.add_trace(go.Bar(x=years, y=v1, name=v1_name,marker_color="#4777d2"))
    fig.add_trace(go.Bar(x=years, y=v2, name=v2_name,marker_color="#d8718b"))

    # Updating layout
    fig.update_layout(
        title=title,
        xaxis_title='Countries',
        yaxis_title='Value',
        barmode='relative',  # Stacked bar chart
        showlegend=True,
        # xaxis={"title": 'title'},
        # yaxis={"title": 'Country'},
        # margin= {"l": 100, "r": 20, "t": 50, "b": 50},
        width= 900,
        height= 700,
        xaxis= {'title': value,'showgrid': False, 'gridcolor': '#75A3BF'},
        yaxis= {'title': title, 'showgrid': True},
        plot_bgcolor= '#f8f8f8',
        paper_bgcolor= '#f8f8f8',
        font= {'family': 'Poppins', 'size': 12, 'color': 'black'},
        legend= {'x': 0.05, 'y': 0.5, 'bgcolor': 'rgba(255, 255, 255, 0.8)', 'bordercolor': 'rgba(0, 0, 0, 0.2)', 'borderwidth': 1},
        margin= {'l': 50, 'r': 50, 'b': 100, 't': 100, 'pad': 4},
    )

    
    

    fig.update_layout(
        annotations=[
            {
                'x': 0,
                'y': -0.14,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'left',
                'yanchor': 'top',
                'text': "Copyright © 2023 Najib Abdullahi",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "left",
            },
            {
                'x': 1.06,
                'y': -0.14,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'right',
                'yanchor': 'top',
                'text': "Source: World Bank",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "right",
            }
        ],
        font=dict(family='Poppins'),
        title_font=dict(size=24, color='#000000'),
        legend_font=dict(size=12),
        legend_itemclick='toggle',
        legend_itemdoubleclick='toggleothers',
        legend_traceorder='normal',
        legend_tracegroupgap=10,
        legend_bordercolor='#000000',
        legend_borderwidth=1,
        legend_bgcolor='#ffffff',
        legend_xanchor='left',
        legend_yanchor='top',
        legend_x=1,
        legend_y=1.05,
        margin=dict(l=50, r=50, t=80, b=50),
    )
    fig.update_yaxes(gridcolor='#c9c1b8')

    fig.show()


In [ ]:
def create_bar_chart_static(dataframe, colors, column, title, countries, value, orientation="v"):
    fig = go.Figure(data=[
    go.Bar(x=dataframe['Country'], y=dataframe[column], marker_color=colors)
])

# Customize the layout
    layout = {
    "title": title,
    "xaxis": {"title": 'Country'},
    "yaxis": {"title": value},
    "margin": {"l": 100, "r": 20, "t": 50, "b": 50},
    "barmode": "group",  # Change to 'group' if you don't want bars stacked
    'autosize': True,
    'width': 900,
    'height': 900,
    'xaxis': {'title': 'Country', 'showgrid': False },
    'yaxis': {'title': value, 'showgrid': True,'gridcolor': '#75A3BF'},
    'plot_bgcolor': '#f8f8f8',
    'paper_bgcolor': '#f8f8f8',
    'font': {'family': 'Poppins', 'size': 12, 'color': 'black'},
    'legend': {'x': 0.05, 'y': 0.5, 'bgcolor': 'rgba(255, 255, 255, 0.8)', 'bordercolor': 'rgba(0, 0, 0, 0.2)', 'borderwidth': 1},
    'margin': {'l': 50, 'r': 50, 'b': 100, 't': 100, 'pad': 4},
}

    fig.update_layout(layout)
    fig.update_layout(
        annotations=[
            {
                'x': 0,
                'y': -0.07,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'left',
                'yanchor': 'top',
                'text': "Copyright © 2023 Najib Abdullahi",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "left",
            },
            {
                'x': 1.06,
                'y': -0.07,
                'xref': 'paper',
                'yref': 'paper',
                'xanchor': 'right',
                'yanchor': 'top',
                'text': "Source: World Bank",
                'showarrow': False,
                'font': dict(size=10, color="#1100FF"),
                'align': "right",
            }
        ],
        font=dict(family='Poppins'),
        title_font=dict(size=15, color='#000000'),
        legend_font=dict(size=12),
        legend_itemclick='toggle',
        legend_itemdoubleclick='toggleothers',
        legend_traceorder='normal',
        legend_tracegroupgap=10,
        legend_bordercolor='#000000',
        legend_borderwidth=1,
        legend_bgcolor='#ffffff',
        legend_xanchor='left',
        legend_yanchor='top',
        legend_x=1,
        legend_y=1.05,
        margin=dict(l=50, r=50, t=80, b=50),
    )

    # Show the chart
    fig.show()

In [ ]:

def create_bar_chart_clustered(df,first_value,second_value,colors,column,title,countries,value,first_value_name,second_value_name):
    # Define the data trace for trace 0
    trace_1 = go.Bar(
    y=df['Country'],
    x=df[first_value],
    name=first_value_name,
    orientation='h',  # Set orientation to horizontal
    marker=dict(color=colors[1])
)
    # Define the data trace for trace 1
    trace_0 = go.Bar(
    y=df['Country'],
    x=df[second_value],
    name=second_value_name,
    orientation='h',  # Set orientation to horizontal
    marker=dict(color=colors[0])
)



# Create the layout
    layout = {
    "title": title,
    "margin": {"l": 100, "r": 20, "t": 50, "b": 50},
    "barmode": "group",
    'autosize': True,
    'width': 900,
    'height': 500,
    'xaxis': {'title': value, 'showgrid': True, 'gridcolor': '#75A3BF'},
    'yaxis': {'title': 'Country', 'showgrid': False},
    'plot_bgcolor': '#f8f8f8',
    'paper_bgcolor': '#f8f8f8',
    'font': {'family': 'Poppins', 'size': 12, 'color': 'black'},
    'legend': {'x': 0.05, 'y': 0.9, 'bgcolor': 'rgba(255, 255, 255, 0.8)', 'bordercolor': 'rgba(0, 0, 0, 0.2)', 'borderwidth': 1},
    'margin': {'l': 50, 'r': 50, 'b': 100, 't': 100, 'pad': 4},
    
}

# Create the figure
    fig = go.Figure(data=[trace_0, trace_1], layout=layout)

# Add annotations
    fig.update_layout(
    annotations=[
        {
            'x': 0,
            'y': -0.06,
            'xref': 'paper',
            'yref': 'paper',
            'xanchor': 'left',
            'yanchor': 'top',
            'text': "Copyright © 2023 Najib Abdullahi",
            'showarrow': False,
            'font': dict(size=10, color="#1100FF"),
            'align': "left",
        },
        {
            'x': 1.06,
            'y': -0.06,
            'xref': 'paper',
            'yref': 'paper',
            'xanchor': 'right',
            'yanchor': 'top',
            'text': "Source: World Bank",
            'showarrow': False,
            'font': dict(size=10, color="#1100FF"),
            'align': "right",
        }
    ],
    font=dict(family='Poppins'),
        title_font=dict(size=15, color='#000000'),
        legend_font=dict(size=12),
        legend_itemclick='toggle',
        legend_itemdoubleclick='toggleothers',
        legend_traceorder='normal',
        legend_tracegroupgap=10,
        legend_bordercolor='#000000',
        legend_borderwidth=1,
        legend_bgcolor='#ffffff',
        legend_xanchor='left',
        legend_yanchor='top',
        legend_x=1,
        legend_y=1,
        margin=dict(l=50, r=50, t=80, b=50),
    )


# Show the chart
    fig.show()

<a id="structure"><h3>Exports of goods and services (current US$)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Exports of goods and services (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Year']>='1996')]
columns=df_selected.columns
create_line_chart_static(df_selected,'Exports of goods and services (current US$)',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Exports of goods and services (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')& (df_selected['Country']!='Djibouti')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"Exports of goods and services (current US$)","Exports of goods and services in 2022",countries,"US$")

In [ ]:

# Extract the required columns
columns = ['Year','Exports of goods and services (current US$)']
df_selected = df[columns]
# Group the data by year and calculate the mean values
df_grouped = df_selected.groupby('Year').mean().round(2).reset_index()
# Extract Required Columns from World Data
cols=['Exports of goods and services (current US$) (World)',]
df_world_columns=world_data[cols]
# print(df_world_columns)
# Mege with Worldwide Data
merged_df = pd.merge(df_grouped, world_data[['Year', 'Exports of goods and services (current US$) (World)']],on='Year', how='outer')

# Rename columns to match the desired format
merged_df.rename(columns={
    'Exports of goods and services (current US$)': 'East Africa Average',
    'Exports of goods and services (current US$) (World)': 'World Average',
}, inplace=True)
# merged_df=merged_df[(merged_df['Year']>='2000')]
columns=merged_df.columns
colors = ['#C10000','#0476D9']
create_line_chart_grouped(merged_df,'Exports of goods and services (current US$)',columns[1:],colors)


In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Exports of goods and services (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Country']!='Sudan')]
avg_arable_land = df_selected.groupby("Country")["Exports of goods and services (current US$)"].median()
countries = df_selected.Country.unique()
plot_horizontal_bar(avg_arable_land, colors,"Average Exports of goods and services (current US$) from 1960-2020",countries,"US$")

<a id="structure"><h3>Imports of goods and services (current US$)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Imports of goods and services (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Year']>='1996')]
columns=df_selected.columns
create_line_chart_static(df_selected,'Imports of goods and services (current US$)',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Imports of goods and services (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')& (df_selected['Country']!='Djibouti')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"Imports of goods and services (current US$)","Imports of goods and services in 2022",countries,"US$")

In [ ]:

# Extract the required columns
columns = ['Year','Imports of goods and services (current US$)']
df_selected = df[columns]
# Group the data by year and calculate the mean values
df_grouped = df_selected.groupby('Year').mean().round(2).reset_index()
# Extract Required Columns from World Data
cols=['Imports of goods and services (current US$) (World)',]
df_world_columns=world_data[cols]
# print(df_world_columns)
# Mege with Worldwide Data
merged_df = pd.merge(df_grouped, world_data[['Year', 'Imports of goods and services (current US$) (World)']],on='Year', how='outer')

# Rename columns to match the desired format
merged_df.rename(columns={
    'Imports of goods and services (current US$)': 'East Africa Average',
    'Imports of goods and services (current US$) (World)': 'World Average',
}, inplace=True)
# merged_df=merged_df[(merged_df['Year']>='2000')]
columns=merged_df.columns
colors = ['#C10000','#0476D9']
create_line_chart_grouped(merged_df,'Imports of goods and services (current US$)',columns[1:],colors,"US$")


In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Imports of goods and services (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Country']!='Sudan')]
avg_arable_land = df_selected.groupby("Country")["Imports of goods and services (current US$)"].median()
countries = df_selected.Country.unique()
plot_horizontal_bar(avg_arable_land, colors,"Average Imports of goods and services (current US$) from 1960-2020",countries,"US$")

In [ ]:
colors=['#57c2af','#c10000']
# Filter for the year 2022
df_2022 = east_africa_countries_data[east_africa_countries_data['Year'] == '2021-01-01']
df_2022=df_2022[(df_2022['Country']!='Djibouti') & (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')]

# Sort the DataFrame by exports in descending order
df_2022 = df_2022.sort_values(by='Imports of goods and services (current US$)', ascending=True)

# Define the data trace for imports
imports=df_2022['Imports of goods and services (current US$)']
exports=df_2022['Exports of goods and services (current US$)']

create_bar_chart_clustered(df_2022,"Imports of goods and services (current US$)","Exports of goods and services (current US$)", colors,"Imports",'Imports and Exports of Each Country in 2022',countries,'US$','Imports','Exports')


<a id="structure"><h3>GDP (current US$)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GDP (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Year']>='1996')]
columns=df_selected.columns
create_line_chart_static(df_selected,'GDP',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GDP (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"GDP (current US$)","GDP in 2022",countries,"US$")

In [ ]:

# Extract the required columns
columns = ['Year','GDP (current US$)']
df_selected = df[columns]
# Group the data by year and calculate the mean values
df_grouped = df_selected.groupby('Year').mean().round(2).reset_index()
# Extract Required Columns from World Data
cols=['GDP (current US$) (World)',]
df_world_columns=world_data[cols]
# print(df_world_columns)
# Mege with Worldwide Data
merged_df = pd.merge(df_grouped, world_data[['Year', 'GDP (current US$) (World)']],on='Year', how='outer')

# Rename columns to match the desired format
merged_df.rename(columns={
    'GDP (current US$)': 'East Africa Average',
    'GDP (current US$) (World)': 'World Average',
}, inplace=True)
# merged_df=merged_df[(merged_df['Year']>='2000')]
columns=merged_df.columns
colors = ['#C10000','#0476D9']
create_line_chart_grouped(merged_df,'GDP (current US$)',columns[1:],colors,"US$")


<a id="structure"><h3>GDP per capita, PPP (current international $)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GDP per capita, PPP (current international $)', 'Country']
df_selected = east_africa_countries_data[columns]
df_selected=df_selected[(df_selected['Year']>='2013')]
columns=df_selected.columns
create_line_chart_static(df_selected,'GDP per Capita adjusted for purchasing power parity (PPP)',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GDP per capita, PPP (current international $)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"GDP per capita, PPP (current international $)","GDP per Capita in 2022 Adjusted for Purchasing Power Parity (PPP)",countries,"US$")

<a id="structure"><h3>GDP growth (annual %)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GDP growth (annual %)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Year']>='1996')]
columns=df_selected.columns
create_line_chart_static(df_selected,'GDP growth (annual %)',columns[1:],colors,"Percentage")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GDP growth (annual %)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"GDP growth (annual %)","GDP growth (annual %) in 2022",countries,"Percentage")

In [ ]:

# Extract the required columns
columns = ['Year','GDP growth (annual %)']
df_selected = df[columns]
# Group the data by year and calculate the mean values
df_grouped = df_selected.groupby('Year').mean().round(2).reset_index()
# Extract Required Columns from World Data
cols=['GDP growth (annual %) (World)',]
df_world_columns=world_data[cols]
# print(df_world_columns)
# Mege with Worldwide Data
merged_df = pd.merge(df_grouped, world_data[['Year', 'GDP growth (annual %) (World)']],on='Year', how='outer')

# Rename columns to match the desired format
merged_df.rename(columns={
    'GDP growth (annual %)': 'East Africa Average',
    'GDP growth (annual %) (World)': 'World Average',
}, inplace=True)
# merged_df=merged_df[(merged_df['Year']>='2000')]
columns=merged_df.columns
colors = ['#C10000','#0476D9']
create_line_chart_grouped(merged_df,'GDP growth (annual %)',columns[1:],colors)


<a id="structure"><h3>External balance on goods and services (current US$)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Net trade in goods and services (BoP, current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
df_selected=df_selected[(df_selected['Year']>='1990')]
columns=df_selected.columns
create_line_chart_static(df_selected,' Net trade in goods and services (BoP, current US$)',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Net trade in goods and services (BoP, current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Country']!='Sudan')]
avg_arable_land = df_selected.groupby("Country")["Net trade in goods and services (BoP, current US$)"].median()
countries = df_selected.Country.unique()
plot_horizontal_bar(avg_arable_land, colors,"Average Trade Deficit from 1960-2020",countries,"US$")

<a id="structure"><h3>GNI (current US$)</h3></a>

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GNI (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Year']>='1996')]
columns=df_selected.columns
create_line_chart_static(df_selected,'GNI (current US$)',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','GNI (current US$)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"GNI (current US$)","GNI In 2022",countries,"US$")

In [ ]:

# Extract the required columns
columns = ['Year','GNI (current US$)']
df_selected = df[columns]
# Group the data by year and calculate the mean values
df_grouped = df_selected.groupby('Year').mean().round(2).reset_index()
# Extract Required Columns from World Data
cols=['GNI (current US$) (World)',]
df_world_columns=world_data[cols]
# print(df_world_columns)
# Mege with Worldwide Data
merged_df = pd.merge(df_grouped, world_data[['Year', 'GNI (current US$) (World)']],on='Year', how='outer')

# Rename columns to match the desired format
merged_df.rename(columns={
    'GNI (current US$)': 'East Africa Average',
    'GNI (current US$) (World)': 'World Average',
}, inplace=True)
# merged_df=merged_df[(merged_df['Year']>='2000')]
columns=merged_df.columns
colors = ['#C10000','#0476D9']
create_line_chart_grouped(merged_df,'GNI (current US$)',columns[1:],colors)


In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Final consumption expenditure (% of GDP)', 'Country']
df_selected = east_africa_countries_data[columns]
# df_selected=df_selected[(df_selected['Year']>='1996')]
columns=df_selected.columns
create_line_chart_static(df_selected,'Final consumption expenditure (% of GDP)',columns[1:],colors,"US$")

In [ ]:
colors = ['#C10000', '#d39b81','#0476D9','#75A3BF','#F2668B','#025E73','#011F26','#026873','#03A688','#D4C2AD','#66796B','#BB0000']
columns = ['Year','Final consumption expenditure (% of GDP)', 'Country']
df_selected = east_africa_countries_data[columns]
# Filter data for the year 2021
data_2022 = df_selected[df_selected['Year'] == '2022-01-01']
data_2022=data_2022[ (df_selected['Country']!='Eritrea')& (df_selected['Country']!='South Sudan')& (df_selected['Country']!='Djibouti')]
countries = data_2022['Country'].unique()
create_bar_chart_static(data_2022,colors,"Final consumption expenditure (% of GDP)","Final consumption expenditure (% of GDP) in 2022",countries,"Percentage")